<a href="https://colab.research.google.com/github/GabrielBG0/Text-Processing-With-Huggingfaces-Transformers/blob/main/Fine_tuning_with_custom_datasets_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Made with help of https://huggingface.co/transformers/custom_datasets.html

In [1]:
# Deleting default sample data folder
!rm -rf /content/sample_data

In [2]:
# Install aditional librarys
!pip install transformers
!pip install datasets

     |████████████████████████████████| 1.8MB 5.2MB/s 
     |████████████████████████████████| 890kB 23.0MB/s 
     |████████████████████████████████| 3.2MB 39.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=d18f914e511bd485d31ac948a8a8af7c4803343c48072cb2fc43ce127eec6196
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 184kB 5.3MB/s 
     |████████████████████████████████| 245kB 8.8MB/s 
     |████████████████████████████████| 20.7MB 1.3MB/s 
     |████████████████████████████████| 112kB 48.8MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [3]:
# Get dataset from imdb
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

--2021-02-24 15:11:52--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  26.5MB/s    in 3.0s    

2021-02-24 15:11:56 (26.5 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [4]:
from pathlib import Path

# Function for separating text and its labels. Gets the name of directory to split as parameter
def read_imdb_split(split_dir):
    split_dir = Path(split_dir)
    texts = []
    labels = []
    # Data is organized in to folders, "pos" and "neg", representing positive and negative reviws
    for label_dir in ["pos", "neg"]:
        for text_file in (split_dir/label_dir).iterdir():
            texts.append(text_file.read_text())
            labels.append(0 if label_dir is "neg" else 1)

    return texts, labels
# Runs the function to divade text and lables from bolth training and testing examples
train_texts, train_labels = read_imdb_split('aclImdb/train')
test_texts, test_labels = read_imdb_split('aclImdb/test')

In [5]:
# Creates a validation dataset use for evaluation and tuning without tainting our test set results
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

In [6]:
# Creates an Bert tokenizer 
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

In [7]:
# Passing text for the tokenizer. Using truncation and paddin as true.
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [8]:
# Turning lables and encoding in to a Dataset object.
import torch

# Class IMDbDataset is a subclass of torch.utils.data.Dataset and inplements __getitem__ and __len__
class IMDbDataset(torch.utils.data.Dataset):
    # Class contructor, gets as parameter encodings and lables
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    # Return ? TODO
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    # Retuen lables lenth
    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

In [9]:
# Checks if there is a gpu available
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [10]:
# Finetuning the model
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification, AdamW

# Sets gpu as processing device if avalable
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Create the model for training
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
model.to(device)
model.train()

# TODO: ?
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(3):
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()

model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

RuntimeError: ignored